In [ ]:
#run only once
%pip install openai chromadb pandas numpy scikit-learn python-dotenv

In [1]:
# ── Cell 2: Imports ─────────────────────────────────────────
import sys
sys.path.append('..')

from src.data_loader  import load_episodes
from src.get_embeddings import add_embeddings_to_df
from src.vector_store import build_collection, get_collection
from src.search       import semantic_search
from src.llm_integeration import interpret_emotional_query
from src.memory       import ConversationMemory, Turn
from scripts.rag_pipeline import run_pipeline, print_results
from src.url_store import add_new_urls

from src.config import PODCASTS_CSV,NEW_URLS_CSV,TRANSCRIPT_EMBEDDINGS_CSV

print('✓ All imports OK')

✓ All imports OK


In [ ]:
# ── Cell 0A: Install (once) ──────────────────────────────────
# %pip install youtube-transcript-api

# ── Cell 0B: Fetch all transcripts ──────────────────────────
import sys
sys.path.append('..')

from src.transcript_fetcher import fetch_all_transcripts, print_summary

# Reads URLs from data/youtube_urls.txt
# Saves results to data/processed/transcripts_df.csv
# Logs errors to logs/transcript_errors.log
df = fetch_all_transcripts(sleep_secs=1.0)

print_summary(df)

# ── Cell 0C: Pass URLs directly (alternative) ────────────────
my_urls = [
    "https://www.youtube.com/watch?v=TbsRU-crgsc",
    "https://www.youtube.com/watch?v=jroF3PH-PTs",
]

df = fetch_all_transcripts(urls=my_urls)
print_summary(df)



In [2]:
import pandas as pd
from pathlib import Path

In [ ]:
# 
df = pd.read_csv(TRANSCRIPT_EMBEDDINGS_CSV)
df = df[df['status']=='success']

In [ ]:
df.to_csv(TRANSCRIPT_EMBEDDINGS_CSV,index=False)

In [ ]:
# Drop common accidental index columns
df = df.drop(columns=[c for c in ["Unnamed: 0", "index", "level_0"] if c in df.columns])

# Also drop any column that starts with "Unnamed"
df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]

df.to_csv(TRANSCRIPT_EMBEDDINGS_CSV, index=False)

In [ ]:
df = pd.read_csv(TRANSCRIPT_EMBEDDINGS_CSV)
df

In [ ]:
urls_df = add_new_urls(PODCASTS_CSV,NEW_URLS_CSV)
urls = urls_df["url"].dropna().astype(str).tolist()

def processed_or_empty(path:Path)->pd.DataFrame:
    if path.exists():
        return pd.read_csv(path)
    else :
        raise FileNotFoundError(
            f"Transcripts file not found :{TRANSCRIPTS_CSV}\n"
            "Extract transcripts first"
        )

#load already processed 
transcripts_df=processed_or_empty(TRANSCRIPTS_CSV)

In [ ]:
 # 3) Find truly new URLs (not already processed)
already_processed = set(transcripts_df["url"].dropna().astype(str).tolist()) if "url" in transcripts_df.columns else set()
new_urls = [u for u in urls if u not in already_processed]

print(f"Total URLs in master list: {len(urls)}")
print(f"New URLs to process: {len(new_urls)}")

if not new_urls:
    print("No new URLs.")

# fetch transcripts only for new urls 
new_transcripts_df = fetch_all_transcripts(urls=new_urls, sleep_secs=1.0, save=False)

# Append and save transcripts
combined = pd.concat([transcripts_df, new_transcripts_df], ignore_index=True)
#(BASE / "data" / "processed").mkdir(parents=True, exist_ok=True)
combined.to_csv(TRANSCRIPTS_CSV, index=False)
print(f"Saved updated transcripts to {TRANSCRIPTS_CSV}")

In [ ]:
combined = pd.read_csv(TRANSCRIPTS_CSV)

In [ ]:
# In your notebook:
from src.get_embeddings import add_embeddings_to_df
from src.data_loader import load_episodes

# Load transcripts
df = pd.read_csv(TRANSCRIPTS_CSV)

# Generate embeddings (automatically chunks long transcripts)
df = add_embeddings_to_df(df, text_col="transcript_clean")

# Save
df.to_csv(TRANSCRIPT_EMBEDDINGS_CSV, index=False)


In [ ]:
# ── Cell 3: Load data with embeddings ────────────────────────────────────────
df = load_episodes()
df[['youtube_title', 'youtube_channel', 'duration_mins', 'word_count']].head()

In [ ]:
from src.vector_store import sync_collection

collection = sync_collection(df)

In [ ]:
df['embedding'].head()

In [ ]:
# ── Cell 4: Build ChromaDB (RUN ONCE ONLY) ──────────────────
collection = build_collection(df, force_rebuild=True)
print(f'Collection has {collection.count()} episodes')

In [ ]:
# ── Cell 5: Load existing collection (USE EVERY SESSION) ────
collection = get_collection()

coerce embedding and new chroma store

In [5]:
# adding new episodes to chromadb
import ast
import numpy as np
import pandas as pd

df = pd.read_csv(TRANSCRIPT_EMBEDDINGS_CSV)

def coerce_embedding(x):
    """Return embedding as List[float] or None if invalid."""
    if x is None:
        return None
    if isinstance(x, float) and pd.isna(x):
        return None

    # already list-like
    if isinstance(x, (list, tuple, np.ndarray)):
        arr = np.array(x, dtype=float)
        return arr.tolist() if arr.ndim == 1 and arr.size > 0 else None

    # stored as string like "[0.1, 0.2, ...]"
    if isinstance(x, str):
        s = x.strip()
        if s.lower() in {"", "nan", "none"}:
            return None
        try:
            parsed = ast.literal_eval(s)
            arr = np.array(parsed, dtype=float)
            return arr.tolist() if arr.ndim == 1 and arr.size > 0 else None
        except Exception:
            return None

    return None

# ---- Clean / validate required columns ----
required_cols = ["video_id", "transcript_clean", "embedding"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"CSV is missing required columns: {missing}")

# ---- Coerce embeddings into real numeric vectors ----
df["embedding"] = df["embedding"].apply(coerce_embedding)

# ---- Drop invalid rows ----
before = len(df)
df = df.dropna(subset=["video_id", "transcript_clean"])
df = df[df["embedding"].notna()].copy()
after = len(df)

print(f"Loaded rows: {before}")
print(f"Valid rows (with embeddings): {after}")
print(f"Dropped rows: {before - after}")

# ---- Ensure consistent embedding dimension ----
dims = df["embedding"].apply(len)
if dims.nunique() != 1:
    common_dim = dims.value_counts().idxmax()
    bad = (dims != common_dim).sum()
    df = df[dims == common_dim].copy()
    print(f"Filtered {bad} rows with non-standard embedding dim. Using dim={common_dim}.")
else:
    print(f"Embedding dim: {dims.iloc[0]}")

# 3. Sync ChromaDB (adds only new episodes)
from src.vector_store import build_collection

# Assuming your signature is build_collection(df, add_only_new: bool)
collection = build_collection(df, True)
print("Chroma sync complete.")

print(f"\n✅ Done! Collection has {collection.count()} episodes")

Loaded rows: 44
Valid rows (with embeddings): 44
Dropped rows: 0
Embedding dim: 1536
deleted old collection
created collection: podcast_episodes
✅ Added 44 episodes to ChromaDB
✓ Collection now has 44 episodes
Chroma sync complete.

✅ Done! Collection has 44 episodes


In [4]:
# Check types
print(type(df.loc[0, "embedding"]))
print(str(df.loc[0, "embedding"])[:120])
print(df["embedding"].isna().sum())

<class 'str'>
[0.02200822673815613, 0.028621132640788954, -0.014038540743058547, 0.043925480296214424, 0.018542302047232322, 0.0137608
0


In [ ]:
import pandas as pd
from src.vector_store import get_collection

# Get what's currently in ChromaDB
collection = get_collection()
existing = collection.get()
existing_ids = set(existing['ids'])

print(f"IDs in ChromaDB: {sorted(existing_ids)}")

# Get what's in your CSV
df = pd.read_csv("data/processed/transcripts_with_embeddings.csv")
csv_ids = set(f"{i+1:03d}" for i in range(len(df)))

print(f"IDs in CSV: {sorted(csv_ids)}")

# Find overlap
overlap = existing_ids & csv_ids
print(f"\nOverlap (already in ChromaDB): {len(overlap)} episodes")
print(f"New episodes: {len(csv_ids - existing_ids)}")

# These should be added:
new_ids = sorted(csv_ids - existing_ids)
print(f"New IDs to add: {new_ids[:10]}...")  # First 10

In [ ]:
from src.vector_store import build_collection
import pandas as pd

df = pd.read_csv(TRANSCRIPT_EMBEDDINGS_CSV)
df = df['embedding'].dropna()

# Nuclear option: delete and rebuild
collection = build_collection(df, force_rebuild=True)

# Should output:
# 🗑️  Deleted old collection
# ✓ Created collection: podcast_episodes
# ⚠️  Row 26 has no embedding — skipping
# ⚠️  Row 27 has no embedding — skipping
# ✅ Added 44 episodes
# ✓ Collection now has 44 episodes

In [ ]:
import pandas as pd

df = pd.read_csv(TRANSCRIPT_EMBEDDINGS_CSV)

print(f"Total rows: {len(df)}")
print(f"\nRows WITH embeddings: {df['embedding'].notna().sum()}")
print(f"Rows WITHOUT embeddings: {df['embedding'].isna().sum()}")

# Check which rows are missing
missing_idx = df[df['embedding'].isna()].index.tolist()
print(f"\nMissing embedding at rows: {missing_idx}")

In [ ]:
print(f"Before: {df['embedding'].notna().sum()}/46 have embeddings")

# Generate embeddings for rows 25 and 26
df = add_embeddings_to_df(df)

print(f"After: {df['embedding'].notna().sum()}/46 have embeddings")

# Save
df.to_csv(TRANSCRIPT_EMBEDDINGS_CSV, index=False)
print("✓ Saved updated CSV")

In [ ]:
# ── Cell 6: Test search ──────────────────────────────────────
test_queries = [
    'I feel like a failure after not getting that job',
    'I am angry and I do not know why',
    'I want to love myself more',
]

for q in test_queries:
    print(f"\n🔍 '{q}'")
    results = semantic_search(q, collection, top_k=2)
    for r in results:
        print(f"  → {r['metadata']['episode_title'][:55]}  ({r['similarity']:.2f})")

In [ ]:
# ── Cell 7: Full RAG pipeline ────────────────────────────────
memory     = ConversationMemory()
user_query = 'I feel really stressed about finding a new job and keep doubting myself'

output = run_pipeline(
    user_query = user_query,
    collection = collection,
    memory     = memory,
    top_k      = 3,
)

print_results(output)

In [ ]:
# ── Cell 8: Check memory ─────────────────────────────────────
print(f'Memory has {len(memory)} turn(s)\n')
print(memory.build_context_string())

In [ ]:
# ── Cell 9: Second turn (memory feeds into LLM) ─────────────
output2 = run_pipeline(
    user_query = 'Now I feel ashamed that I am not handling this better',
    collection = collection,
    memory     = memory,
    top_k      = 2,
)

print_results(output2)
print(f'\nMemory now has {len(memory)} turns')

In [ ]:
# ── Cell 10: Reset memory ────────────────────────────────────
memory.clear()
print(f'Cleared. Turns: {len(memory)}')